# TaskB

TaskB1. 

This document describes a data model that uses embedded documents to describe relationships between connected data. 
Consider the TaskB that maps climatedata and multiple firedata relationships. 
We chose embedding over referencing because if we need to view many data entities in context of another and in single operation.

In other words, in this assignment case, our primary target is to create a model to help users to read data more conveniently. 
Thus some disadvantages of embedded are not the priority thing we need to consider. For instance, we do not need to consider if the data growth.
Because the data scale would not be increasing. 

Over all, we think the embedded model is the better model to operate the following taskB.

The model should be like:

In [306]:
import pymongo
from pprint import pprint
from pymongo import MongoClient
import multiprocessing
from multiprocessing import Pool

TaskB2~B3

I will read the climate data (climatePart-1.csv) and fire data (firePart-1.csv) form taskA which has been imported to the mongodb already. 
I will create a new collection "climate_fire" based on the document model I have designed in Task B1

In [357]:
client = MongoClient()       #Initial client
db = client.fit5148_db       #Use the same fit5148_db Database
climate = db.climate         #read the climate data (climatePart-1.csv) form mongoDB
fire = db.fire               #read the fire data (firePart-1.csv) form mongoDB
climate_fire = db.climate_fire       #create a new collection "climate_fire"

In [358]:
'''
Use for loop to match the same Date in fire data and climate data
Put the fire data into climate data which has the same date
'''

results = []                              
for eachClimate in climate.find({}):           
        results.append(eachClimate)
for eachRecord in results:
        eachRecord["climate_fire"] = []    
        for eachFire in fire.find({"Date" : eachRecord["Date"]}):
            eachRecord["climate_fire"].append(eachFire)

In [359]:
result = climate_fire.insert_many(results)     #insert to mongoDB

In [360]:
climatefire = db.climate_fire                  #read the new collection from mongoDB

Task.B4.2

Find climate data on 15th December 2017

In [361]:
pprint(climatefire.find_one({"Date":"2017-12-15"}))

{'Air Temperature(Celcius)': 18,
 'Date': '2017-12-15',
 'MAX': '74.5*',
 'MIN': 53.1,
 'Max Wind Speed': 14,
 'Precipitation': '0.00I',
 'Relative Humidity': 52,
 'Station': 948702,
 'WindSpeed  (knots)': 7.1,
 '_id': ObjectId('5af680fbd4b2e7c4c47a9209'),
 'climate_fire': [{'Confidence': 92,
                   'Date': '2017-12-15',
                   'Datetime': '2017-12-15T13:17:17',
                   'Latitude': -37.627,
                   'Longitude': 149.33,
                   'Power': 23.1,
                   'Surface Temperature (Celcius)': 42,
                   'Surface Temperature (kelvin)': 315.9,
                   '_id': ObjectId('5af68101d4b2e7c4c47a9236')},
                  {'Confidence': 78,
                   'Date': '2017-12-15',
                   'Datetime': '2017-12-15T13:17:17',
                   'Latitude': -37.658,
                   'Longitude': 149.339,
                   'Power': 17.3,
                   'Surface Temperature (Celcius)': 36,
               

I used rr partition and linear search to partition the data-set and find the date we want, then implement it in parallel

In [266]:
def task2_linear_seach(data, date):
    """
        Perform linear search on given dataset
        
        Parameters:
        date: the dataset to be searched
        date: the date(can be a range) used for searching
        
        Return:
        result: a tuple containing the index of the matched record and the query result
        """
    result = []
    for i in data:
        if i["Date"] == date:
            result.append(i)
    return result


def task2_rr_partition(data,n):
    """
        Perform a simple round robin partition on the given data set
        
        Parameters:
        data: the dataset to be partitioned, which is a list
        n: the number of groups that the dataset will be divided into
        
        Return:
        result: the partitioned subset of the dataset
        """
    result = []
    for i in range(n):
        result.append([])
    for index,element in enumerate(data):
        index_bin = index%n
        result[index_bin].append(element)
    return result

In [278]:
def task2_parallel_search(data,query):
    """
        A method doing parallel search on a given dataset ,
        when given a search clue like a single key or a range for certain column value
        
        Parameters:
        data: the dataset to be searched, which is a list
        query: a query record
        n_processer: the number of processor to parallize the search job
        
        Return:
        results: the list of all search results in all processors
        """
    result = []
    results = []
    pool = Pool(processes=3)
    datasets = task2_rr_partition(data, 3)
    for partition in datasets:
        result = pool.apply_async(task2_linear_seach, args=(partition,query))
        output = result.get()
        results.append(output)
        
    pool.close()
    return results

In [279]:
task2_parallel_search(climatefire.find({}), "2017-12-15")

[[{'Air Temperature(Celcius)': 18,
   'Date': '2017-12-15',
   'MAX': '74.5*',
   'MIN': 53.1,
   'Max Wind Speed': 14,
   'Precipitation': '0.00I',
   'Relative Humidity': 52,
   'Station': 948702,
   'WindSpeed  (knots)': 7.1,
   '_id': ObjectId('5af680fbd4b2e7c4c47a9209'),
   'climate_fire': [{'Confidence': 92,
     'Date': '2017-12-15',
     'Datetime': '2017-12-15T13:17:17',
     'Latitude': -37.627,
     'Longitude': 149.33,
     'Power': 23.1,
     'Surface Temperature (Celcius)': 42,
     'Surface Temperature (kelvin)': 315.9,
     '_id': ObjectId('5af68101d4b2e7c4c47a9236')},
    {'Confidence': 78,
     'Date': '2017-12-15',
     'Datetime': '2017-12-15T13:17:17',
     'Latitude': -37.658,
     'Longitude': 149.339,
     'Power': 17.3,
     'Surface Temperature (Celcius)': 36,
     'Surface Temperature (kelvin)': 309.8,
     '_id': ObjectId('5af68101d4b2e7c4c47a9237')},
    {'Confidence': 51,
     'Date': '2017-12-15',
     'Datetime': '2017-12-15T13:17:17',
     'Latitude': -

TaskB.4.3

Find the latitude, longitude and confidence when the surface temperature (°C) was between 65 °C and 100 °C.

In [216]:
results = climatefire.find({"climate_fire.Surface Temperature (Celcius)": {"$gte": 65, "$lte":100}}, {"climate_fire.Latitude":1,"climate_fire.Longitude":1,"climate_fire.Confidence":1})
for doc in results:
    pprint(doc)

{'_id': ObjectId('5af680fbd4b2e7c4c47a90f1'),
 'climate_fire': [{'Confidence': 100,
                   'Latitude': -37.223,
                   'Longitude': 147.9431},
                  {'Confidence': 100,
                   'Latitude': -37.2252,
                   'Longitude': 147.9363},
                  {'Confidence': 94,
                   'Latitude': -37.2284,
                   'Longitude': 147.9187},
                  {'Confidence': 97,
                   'Latitude': -37.6572,
                   'Longitude': 142.0703},
                  {'Confidence': 81,
                   'Latitude': -37.6592,
                   'Longitude': 142.058},
                  {'Confidence': 55,
                   'Latitude': -36.2544,
                   'Longitude': 148.0353},
                  {'Confidence': 74,
                   'Latitude': -37.2128,
                   'Longitude': 147.9308},
                  {'Confidence': 54,
                   'Latitude': -37.2197,
                   'Longitude

                  {'Confidence': 77,
                   'Latitude': -36.9151,
                   'Longitude': 142.0798},
                  {'Confidence': 56,
                   'Latitude': -37.8089,
                   'Longitude': 145.9555},
                  {'Confidence': 85,
                   'Latitude': -37.9112,
                   'Longitude': 143.6025},
                  {'Confidence': 62,
                   'Latitude': -37.9009,
                   'Longitude': 143.6217},
                  {'Confidence': 69,
                   'Latitude': -37.7836,
                   'Longitude': 142.9437},
                  {'Confidence': 70, 'Latitude': -37.7583, 'Longitude': 143.41},
                  {'Confidence': 99,
                   'Latitude': -37.7236,
                   'Longitude': 143.3995},
                  {'Confidence': 76,
                   'Latitude': -37.5402,
                   'Longitude': 143.2559},
                  {'Confidence': 93,
                   'Latitude': -37.

                   'Latitude': -36.7722,
                   'Longitude': 141.5514},
                  {'Confidence': 87,
                   'Latitude': -36.0973,
                   'Longitude': 143.4279},
                  {'Confidence': 66,
                   'Latitude': -37.0081,
                   'Longitude': 141.7654},
                  {'Confidence': 53,
                   'Latitude': -36.3539,
                   'Longitude': 142.1894},
                  {'Confidence': 100,
                   'Latitude': -36.0967,
                   'Longitude': 141.7587},
                  {'Confidence': 73,
                   'Latitude': -36.4204,
                   'Longitude': 141.3765},
                  {'Confidence': 64,
                   'Latitude': -36.2889,
                   'Longitude': 141.7692},
                  {'Confidence': 74,
                   'Latitude': -35.4439,
                   'Longitude': 143.2005},
                  {'Confidence': 81,
                   'Latitude': 

                   'Latitude': -37.3359,
                   'Longitude': 149.3816},
                  {'Confidence': 66,
                   'Latitude': -37.3526,
                   'Longitude': 149.3972},
                  {'Confidence': 97,
                   'Latitude': -37.3191,
                   'Longitude': 149.366},
                  {'Confidence': 100,
                   'Latitude': -36.7408,
                   'Longitude': 147.1243},
                  {'Confidence': 92, 'Latitude': -36.762, 'Longitude': 147.12},
                  {'Confidence': 68,
                   'Latitude': -37.3344,
                   'Longitude': 149.3933},
                  {'Confidence': 74,
                   'Latitude': -37.3283,
                   'Longitude': 149.368},
                  {'Confidence': 61,
                   'Latitude': -37.3511,
                   'Longitude': 149.4088},
                  {'Confidence': 76,
                   'Latitude': -37.3085,
                   'Longitude': 1

                   'Longitude': 141.0622},
                  {'Confidence': 100,
                   'Latitude': -36.91,
                   'Longitude': 141.2705},
                  {'Confidence': 59,
                   'Latitude': -37.7576,
                   'Longitude': 142.465},
                  {'Confidence': 77,
                   'Latitude': -36.7377,
                   'Longitude': 141.3299},
                  {'Confidence': 57,
                   'Latitude': -38.0505,
                   'Longitude': 142.4791},
                  {'Confidence': 71,
                   'Latitude': -36.6667,
                   'Longitude': 143.7526},
                  {'Confidence': 89,
                   'Latitude': -35.8071,
                   'Longitude': 142.7318},
                  {'Confidence': 52,
                   'Latitude': -37.7851,
                   'Longitude': 141.4593},
                  {'Confidence': 97,
                   'Latitude': -36.7787,
                   'Longitude': 14

...

                  {'Confidence': 91,
                   'Latitude': -36.3961,
                   'Longitude': 140.9781},
                  {'Confidence': 76,
                   'Latitude': -38.4025,
                   'Longitude': 146.6501},
                  {'Confidence': 100,
                   'Latitude': -36.7708,
                   'Longitude': 145.1908},
                  {'Confidence': 83,
                   'Latitude': -36.6173,
                   'Longitude': 142.1868},
                  {'Confidence': 78,
                   'Latitude': -37.8701,
                   'Longitude': 142.8066},
                  {'Confidence': 98, 'Latitude': -37.758, 'Longitude': 143.179},
                  {'Confidence': 94,
                   'Latitude': -37.7609,
                   'Longitude': 143.1633},
                  {'Confidence': 89,
                   'Latitude': -36.5116,
                   'Longitude': 144.562},
                  {'Confidence': 100,
                   'Latitude': -36

                  {'Confidence': 64,
                   'Latitude': -36.3452,
                   'Longitude': 145.8969},
                  {'Confidence': 71,
                   'Latitude': -36.686,
                   'Longitude': 143.8575},
                  {'Confidence': 71,
                   'Latitude': -36.3617,
                   'Longitude': 141.0269},
                  {'Confidence': 56, 'Latitude': -37.785, 'Longitude': 143.092},
                  {'Confidence': 77,
                   'Latitude': -36.6118,
                   'Longitude': 143.5275},
                  {'Confidence': 62,
                   'Latitude': -37.8254,
                   'Longitude': 143.3904},
                  {'Confidence': 78,
                   'Latitude': -36.0786,
                   'Longitude': 144.229},
                  {'Confidence': 75,
                   'Latitude': -36.1369,
                   'Longitude': 141.8122},
                  {'Confidence': 54,
                   'Latitude': -36.29

                   'Longitude': 141.3188},
                  {'Confidence': 94,
                   'Latitude': -37.4357,
                   'Longitude': 142.7813},
                  {'Confidence': 76,
                   'Latitude': -37.3233,
                   'Longitude': 143.8061},
                  {'Confidence': 82,
                   'Latitude': -36.2646,
                   'Longitude': 141.0021},
                  {'Confidence': 60,
                   'Latitude': -35.7421,
                   'Longitude': 142.5634},
                  {'Confidence': 87,
                   'Latitude': -37.4479,
                   'Longitude': 142.7671},
                  {'Confidence': 54,
                   'Latitude': -37.459,
                   'Longitude': 142.9198},
                  {'Confidence': 60,
                   'Latitude': -37.4304,
                   'Longitude': 143.1275},
                  {'Confidence': 75,
                   'Latitude': -37.3333,
                   'Longitude': 1

                  {'Confidence': 78, 'Latitude': -36.4348, 'Longitude': 141.3},
                  {'Confidence': 71,
                   'Latitude': -36.3638,
                   'Longitude': 143.9086},
                  {'Confidence': 63,
                   'Latitude': -36.3542,
                   'Longitude': 141.0935},
                  {'Confidence': 79,
                   'Latitude': -36.4277,
                   'Longitude': 141.3047},
                  {'Confidence': 78,
                   'Latitude': -36.3362,
                   'Longitude': 141.4973},
                  {'Confidence': 57,
                   'Latitude': -35.6785,
                   'Longitude': 143.5077},
                  {'Confidence': 68,
                   'Latitude': -36.6755,
                   'Longitude': 141.5039},
                  {'Confidence': 74,
                   'Latitude': -35.4591,
                   'Longitude': 142.062},
                  {'Confidence': 71,
                   'Latitude': -34.50

                  {'Confidence': 100,
                   'Latitude': -37.237,
                   'Longitude': 142.141},
                  {'Confidence': 73, 'Latitude': -37.238, 'Longitude': 142.13},
                  {'Confidence': 88,
                   'Latitude': -38.002,
                   'Longitude': 144.514}]}
{'_id': ObjectId('5af680fbd4b2e7c4c47a91b6'),
 'climate_fire': [{'Confidence': 70, 'Latitude': -37.497, 'Longitude': 147.266},
                  {'Confidence': 86, 'Latitude': -37.57, 'Longitude': 148.034},
                  {'Confidence': 56, 'Latitude': -37.559, 'Longitude': 148.037},
                  {'Confidence': 58, 'Latitude': -37.379, 'Longitude': 148.126},
                  {'Confidence': 62, 'Latitude': -37.379, 'Longitude': 148.132},
                  {'Confidence': 100,
                   'Latitude': -37.434,
                   'Longitude': 148.123},
                  {'Confidence': 100,
                   'Latitude': -37.396,
                   'Longitude': 

TaskB.4.4

Find surface temperature (°C), air temperature (°C), relative humidity and maximum wind speed on 15th and 16th of December 2017.

In [397]:
results = climatefire.find({"Date": {"$gte": "2017-12-15", "$lte": "2017-12-16"}}, {"_id": 0, "Air Temperature(Celcius)":1, "Relative Humidity":1, "Max Wind Speed":1, "Date":1, "climate_fire.Surface Temperature (Celcius)":1 })
for doc in results:
    pprint(doc)

{'Air Temperature(Celcius)': 18,
 'Date': '2017-12-15',
 'Max Wind Speed': 14,
 'Relative Humidity': 52,
 'climate_fire': [{'Surface Temperature (Celcius)': 42},
                  {'Surface Temperature (Celcius)': 36},
                  {'Surface Temperature (Celcius)': 38},
                  {'Surface Temperature (Celcius)': 40}]}
{'Air Temperature(Celcius)': 18,
 'Date': '2017-12-16',
 'Max Wind Speed': 13,
 'Relative Humidity': 53.7,
 'climate_fire': [{'Surface Temperature (Celcius)': 43},
                  {'Surface Temperature (Celcius)': 33},
                  {'Surface Temperature (Celcius)': 54},
                  {'Surface Temperature (Celcius)': 73},
                  {'Surface Temperature (Celcius)': 55},
                  {'Surface Temperature (Celcius)': 75},
                  {'Surface Temperature (Celcius)': 55},
                  {'Surface Temperature (Celcius)': 66},
                  {'Surface Temperature (Celcius)': 56},
                  {'Surface Temperature (Celci

TaskB.4.5

Find datetime, air temperature (°C), surface temperature (°C) and confidence when the confidence is between 80 and 100.

In [203]:
results = climatefire.find({"climate_fire.Confidence": {"$gte": 80, "$lte": 100}}, {"_id": 0, "Air Temperature(Celcius)":1, "climate_fire.Confidence":1, "climate_fire.Datetime":1, "climate_fire.Surface Temperature (Celcius)":1})
for doc in results:
    pprint(doc)

{'Air Temperature(Celcius)': 20,
 'climate_fire': [{'Confidence': 85,
                   'Datetime': '2017-03-06T05:06:20',
                   'Surface Temperature (Celcius)': 59},
                  {'Confidence': 87,
                   'Datetime': '2017-03-06T05:06:30',
                   'Surface Temperature (Celcius)': 62}]}
{'Air Temperature(Celcius)': 19,
 'climate_fire': [{'Confidence': 88,
                   'Datetime': '2017-03-07T04:16:10',
                   'Surface Temperature (Celcius)': 64}]}
{'Air Temperature(Celcius)': 23,
 'climate_fire': [{'Confidence': 86,
                   'Datetime': '2017-03-09T13:23:40',
                   'Surface Temperature (Celcius)': 41},
                  {'Confidence': 54,
                   'Datetime': '2017-03-09T03:57:00',
                   'Surface Temperature (Celcius)': 44},
                  {'Confidence': 78,
                   'Datetime': '2017-03-09T03:56:50',
                   'Surface Temperature (Celcius)': 55}]}
{'Air Temp

                  {'Confidence': 100,
                   'Datetime': '2017-04-03T03:53:50',
                   'Surface Temperature (Celcius)': 93},
                  {'Confidence': 71,
                   'Datetime': '2017-04-03T03:53:50',
                   'Surface Temperature (Celcius)': 45},
                  {'Confidence': 96,
                   'Datetime': '2017-04-03T03:54:00',
                   'Surface Temperature (Celcius)': 78},
                  {'Confidence': 69,
                   'Datetime': '2017-04-03T03:53:50',
                   'Surface Temperature (Celcius)': 44},
                  {'Confidence': 68,
                   'Datetime': '2017-04-03T03:53:40',
                   'Surface Temperature (Celcius)': 44},
                  {'Confidence': 85,
                   'Datetime': '2017-04-03T03:53:30',
                   'Surface Temperature (Celcius)': 59},
                  {'Confidence': 66,
                   'Datetime': '2017-04-03T03:53:30',
                   '

                   'Datetime': '2017-04-04T04:35:00',
                   'Surface Temperature (Celcius)': 95},
                  {'Confidence': 70,
                   'Datetime': '2017-04-04T04:34:30',
                   'Surface Temperature (Celcius)': 51},
                  {'Confidence': 83,
                   'Datetime': '2017-04-04T04:34:50',
                   'Surface Temperature (Celcius)': 57},
                  {'Confidence': 89,
                   'Datetime': '2017-04-04T04:34:20',
                   'Surface Temperature (Celcius)': 65},
                  {'Confidence': 79,
                   'Datetime': '2017-04-04T04:34:20',
                   'Surface Temperature (Celcius)': 52},
                  {'Confidence': 54,
                   'Datetime': '2017-04-04T04:34:20',
                   'Surface Temperature (Celcius)': 40},
                  {'Confidence': 96,
                   'Datetime': '2017-04-04T04:34:20',
                   'Surface Temperature (Celcius)': 78},
 

                   'Datetime': '2017-04-06T04:20:40',
                   'Surface Temperature (Celcius)': 53},
                  {'Confidence': 85,
                   'Datetime': '2017-04-06T04:20:40',
                   'Surface Temperature (Celcius)': 60},
                  {'Confidence': 52,
                   'Datetime': '2017-04-06T04:20:40',
                   'Surface Temperature (Celcius)': 47},
                  {'Confidence': 75,
                   'Datetime': '2017-04-06T04:20:40',
                   'Surface Temperature (Celcius)': 64},
                  {'Confidence': 71,
                   'Datetime': '2017-04-06T04:20:40',
                   'Surface Temperature (Celcius)': 47},
                  {'Confidence': 87,
                   'Datetime': '2017-04-06T04:20:40',
                   'Surface Temperature (Celcius)': 62},
                  {'Confidence': 90,
                   'Datetime': '2017-04-06T04:20:40',
                   'Surface Temperature (Celcius)': 67},
 

                  {'Confidence': 84,
                   'Datetime': '2017-04-12T03:47:50',
                   'Surface Temperature (Celcius)': 58},
                  {'Confidence': 84,
                   'Datetime': '2017-04-12T03:47:30',
                   'Surface Temperature (Celcius)': 58},
                  {'Confidence': 70,
                   'Datetime': '2017-04-12T03:47:30',
                   'Surface Temperature (Celcius)': 45},
                  {'Confidence': 100,
                   'Datetime': '2017-04-12T03:47:30',
                   'Surface Temperature (Celcius)': 106},
                  {'Confidence': 83,
                   'Datetime': '2017-04-12T03:47:20',
                   'Surface Temperature (Celcius)': 56},
                  {'Confidence': 84,
                   'Datetime': '2017-04-12T03:47:20',
                   'Surface Temperature (Celcius)': 57},
                  {'Confidence': 65,
                   'Datetime': '2017-04-12T03:46:50',
                   

...

                   'Surface Temperature (Celcius)': 53},
                  {'Confidence': 99,
                   'Datetime': '2017-05-10T04:08:10',
                   'Surface Temperature (Celcius)': 85}]}
{'Air Temperature(Celcius)': 9,
 'climate_fire': [{'Confidence': 90,
                   'Datetime': '2017-05-11T04:57:20',
                   'Surface Temperature (Celcius)': 67},
                  {'Confidence': 96,
                   'Datetime': '2017-05-11T04:57:20',
                   'Surface Temperature (Celcius)': 79},
                  {'Confidence': 88,
                   'Datetime': '2017-05-11T04:54:00',
                   'Surface Temperature (Celcius)': 63},
                  {'Confidence': 84,
                   'Datetime': '2017-05-11T04:50:40',
                   'Surface Temperature (Celcius)': 78},
                  {'Confidence': 89,
                   'Datetime': '2017-05-11T04:51:00',
                   'Surface Temperature (Celcius)': 65},
                  {'Co

                  {'Confidence': 52,
                   'Datetime': '2017-05-15T04:26:20',
                   'Surface Temperature (Celcius)': 48},
                  {'Confidence': 100,
                   'Datetime': '2017-05-15T04:26:20',
                   'Surface Temperature (Celcius)': 111},
                  {'Confidence': 89,
                   'Datetime': '2017-05-15T04:26:20',
                   'Surface Temperature (Celcius)': 65},
                  {'Confidence': 70,
                   'Datetime': '2017-05-15T04:26:20',
                   'Surface Temperature (Celcius)': 45},
                  {'Confidence': 63,
                   'Datetime': '2017-05-15T04:26:20',
                   'Surface Temperature (Celcius)': 42},
                  {'Confidence': 64,
                   'Datetime': '2017-05-15T04:26:20',
                   'Surface Temperature (Celcius)': 42},
                  {'Confidence': 70,
                   'Datetime': '2017-05-15T04:26:20',
                   

                  {'Confidence': 97,
                   'Datetime': '2017-06-20T04:02:00',
                   'Surface Temperature (Celcius)': 80},
                  {'Confidence': 100,
                   'Datetime': '2017-06-20T04:02:00',
                   'Surface Temperature (Celcius)': 89},
                  {'Confidence': 100,
                   'Datetime': '2017-06-20T04:02:00',
                   'Surface Temperature (Celcius)': 88}]}
{'Air Temperature(Celcius)': 5,
 'climate_fire': [{'Confidence': 78,
                   'Datetime': '2017-06-30T04:41:25',
                   'Surface Temperature (Celcius)': 44},
                  {'Confidence': 81,
                   'Datetime': '2017-06-30T00:27:19',
                   'Surface Temperature (Celcius)': 48},
                  {'Confidence': 78,
                   'Datetime': '2017-06-30T04:41:25',
                   'Surface Temperature (Celcius)': 44},
                  {'Confidence': 75,
                   'Datetime': '2017-06-

                   'Datetime': '2017-10-03T03:58:15',
                   'Surface Temperature (Celcius)': 55},
                  {'Confidence': 71,
                   'Datetime': '2017-10-03T03:58:14',
                   'Surface Temperature (Celcius)': 50},
                  {'Confidence': 100,
                   'Datetime': '2017-10-03T03:58:14',
                   'Surface Temperature (Celcius)': 90},
                  {'Confidence': 88,
                   'Datetime': '2017-10-03T03:58:14',
                   'Surface Temperature (Celcius)': 65},
                  {'Confidence': 87,
                   'Datetime': '2017-10-03T03:58:14',
                   'Surface Temperature (Celcius)': 59},
                  {'Confidence': 54,
                   'Datetime': '2017-10-03T01:22:44',
                   'Surface Temperature (Celcius)': 31},
                  {'Confidence': 73,
                   'Datetime': '2017-10-03T01:22:44',
                   'Surface Temperature (Celcius)': 39}]}

                   'Surface Temperature (Celcius)': 41}]}
{'Air Temperature(Celcius)': 15,
 'climate_fire': [{'Confidence': 77,
                   'Datetime': '2017-12-08T13:11:44',
                   'Surface Temperature (Celcius)': 36},
                  {'Confidence': 100,
                   'Datetime': '2017-12-08T13:11:44',
                   'Surface Temperature (Celcius)': 54},
                  {'Confidence': 100,
                   'Datetime': '2017-12-08T13:11:44',
                   'Surface Temperature (Celcius)': 71},
                  {'Confidence': 100,
                   'Datetime': '2017-12-08T13:11:43',
                   'Surface Temperature (Celcius)': 52},
                  {'Confidence': 56,
                   'Datetime': '2017-12-08T13:11:43',
                   'Surface Temperature (Celcius)': 40}]}
{'Air Temperature(Celcius)': 17,
 'climate_fire': [{'Confidence': 87,
                   'Datetime': '2017-12-09T00:14:34',
                   'Surface Temperature (

TaskB.4.6

Find top 10 records with highest surface temperature (°C).

sort in pymongo, takes key and direction as parameters.

So if you want to sort by, let's say, id then you should .sort("_id", 1)

In [398]:
results = climatefire.aggregate([
{"$unwind": "$climate_fire"},
{"$sort" : { "climate_fire.Surface Temperature (Celcius)" : -1}},
{"$limit":10}
])
for doc in results:
    pprint(doc)

{'Air Temperature(Celcius)': 15,
 'Date': '2017-04-18',
 'MAX': '69.8*',
 'MIN': 46.8,
 'Max Wind Speed': 9.9,
 'Precipitation': '0.00I',
 'Relative Humidity': 56.1,
 'Station': 948701,
 'WindSpeed  (knots)': 5.1,
 '_id': ObjectId('5af680fbd4b2e7c4c47a9118'),
 'climate_fire': {'Confidence': 100,
                  'Date': '2017-04-18',
                  'Datetime': '2017-04-18T04:52:00',
                  'Latitude': -38.1665,
                  'Longitude': 143.062,
                  'Power': 239.8,
                  'Surface Temperature (Celcius)': 124,
                  'Surface Temperature (kelvin)': 397.5,
                  '_id': ObjectId('5af68101d4b2e7c4c47a96c3')}}
{'Air Temperature(Celcius)': 16,
 'Date': '2017-04-04',
 'MAX': '73.4*',
 'MIN': 46.8,
 'Max Wind Speed': 12,
 'Precipitation': '0.00I',
 'Relative Humidity': 47.5,
 'Station': 948701,
 'WindSpeed  (knots)': 5.4,
 '_id': ObjectId('5af680fbd4b2e7c4c47a910b'),
 'climate_fire': {'Confidence': 100,
                  'Date

TaskB.4.7

Find the number of fire in each day. You are required to only display total number of fire
and the date in the output.

In [148]:
results = climatefire.aggregate([
    {"$project" : { "Date": "$climate_fire.Date", "_id" : "$_id"}},
    {"$unwind" : "$Date"},
    {"$group":{ "_id": {"date": "$Date"}, "count":{"$sum":1}}}
])
for doc in results:
    pprint(doc)

{'_id': {'date': '2017-12-27'}, 'count': 4}
{'_id': {'date': '2017-12-24'}, 'count': 1}
{'_id': {'date': '2017-12-15'}, 'count': 4}
{'_id': {'date': '2017-12-14'}, 'count': 1}
{'_id': {'date': '2017-12-12'}, 'count': 1}
{'_id': {'date': '2017-12-10'}, 'count': 2}
{'_id': {'date': '2017-12-09'}, 'count': 4}
{'_id': {'date': '2017-11-29'}, 'count': 8}
{'_id': {'date': '2017-11-28'}, 'count': 1}
{'_id': {'date': '2017-11-23'}, 'count': 5}
{'_id': {'date': '2017-11-22'}, 'count': 2}
{'_id': {'date': '2017-11-14'}, 'count': 3}
{'_id': {'date': '2017-11-13'}, 'count': 5}
{'_id': {'date': '2017-11-11'}, 'count': 4}
{'_id': {'date': '2017-11-09'}, 'count': 10}
{'_id': {'date': '2017-11-05'}, 'count': 4}
{'_id': {'date': '2017-10-27'}, 'count': 5}
{'_id': {'date': '2017-10-18'}, 'count': 6}
{'_id': {'date': '2017-10-16'}, 'count': 1}
{'_id': {'date': '2017-10-15'}, 'count': 3}
{'_id': {'date': '2017-10-10'}, 'count': 3}
{'_id': {'date': '2017-10-09'}, 'count': 1}
{'_id': {'date': '2017-10-17'},

TaskB.4.8

Find the average surface temperature (°C) for each day. You are required to only display
average surface temperature (°C) and the date in the output.

In [379]:
results = climatefire.aggregate([
    {"$project" : { "Date": "$climate_fire.Date", "Surface Temperature": "$climate_fire.Surface Temperature (Celcius)"}},
    {"$unwind" : "$Date"},
    {"$unwind" : "$Surface Temperature"},
    {"$group":{ "_id": {"date": "$Date"},"avgSurfaceTem":{"$avg": "$Surface Temperature"}}}
])
for doc in results:
    pprint(doc)

{'_id': {'date': '2017-12-27'}, 'avgSurfaceTem': 62.75}
{'_id': {'date': '2017-12-24'}, 'avgSurfaceTem': 32.0}
{'_id': {'date': '2017-12-15'}, 'avgSurfaceTem': 39.0}
{'_id': {'date': '2017-12-14'}, 'avgSurfaceTem': 70.0}
{'_id': {'date': '2017-12-12'}, 'avgSurfaceTem': 44.0}
{'_id': {'date': '2017-12-10'}, 'avgSurfaceTem': 46.0}
{'_id': {'date': '2017-12-09'}, 'avgSurfaceTem': 58.25}
{'_id': {'date': '2017-11-22'}, 'avgSurfaceTem': 61.5}
{'_id': {'date': '2017-11-29'}, 'avgSurfaceTem': 60.625}
{'_id': {'date': '2017-11-28'}, 'avgSurfaceTem': 42.0}
{'_id': {'date': '2017-11-09'}, 'avgSurfaceTem': 61.3}
{'_id': {'date': '2017-11-23'}, 'avgSurfaceTem': 58.8}
{'_id': {'date': '2017-11-14'}, 'avgSurfaceTem': 52.0}
{'_id': {'date': '2017-11-13'}, 'avgSurfaceTem': 47.0}
{'_id': {'date': '2017-11-11'}, 'avgSurfaceTem': 46.25}
{'_id': {'date': '2017-11-05'}, 'avgSurfaceTem': 58.5}
{'_id': {'date': '2017-10-27'}, 'avgSurfaceTem': 50.4}
{'_id': {'date': '2017-10-18'}, 'avgSurfaceTem': 52.16666666

In [382]:
results = climatefire.find({})
for doc in results:
    pprint(doc)

{'Air Temperature(Celcius)': 19,
 'Date': '2016-12-31',
 'MAX': '72.0*',
 'MIN': '61.9*',
 'Max Wind Speed': 11.1,
 'Precipitation': '0.00I',
 'Relative Humidity': 56.8,
 'Station': 948700,
 'WindSpeed  (knots)': 7.9,
 '_id': ObjectId('5af680fbd4b2e7c4c47a90ad'),
 'climate_fire': []}
{'Air Temperature(Celcius)': 24,
 'Date': '2017-01-04',
 'MAX': '87.3*',
 'MIN': 54,
 'Max Wind Speed': 14,
 'Precipitation': '0.00I',
 'Relative Humidity': 61.6,
 'Station': 948700,
 'WindSpeed  (knots)': 7.7,
 '_id': ObjectId('5af680fbd4b2e7c4c47a90ae'),
 'climate_fire': []}
{'Air Temperature(Celcius)': 16,
 'Date': '2017-01-03',
 'MAX': '68.4*',
 'MIN': '54.3*',
 'Max Wind Speed': 15,
 'Precipitation': '0.00G',
 'Relative Humidity': 53.6,
 'Station': 948700,
 'WindSpeed  (knots)': 8.1,
 '_id': ObjectId('5af680fbd4b2e7c4c47a90af'),
 'climate_fire': []}
{'Air Temperature(Celcius)': 15,
 'Date': '2017-01-02',
 'MAX': '72.0*',
 'MIN': 58.3,
 'Max Wind Speed': 13,
 'Precipitation': '0.02G',
 'Relative Humidi

 'WindSpeed  (knots)': 10.9,
 '_id': ObjectId('5af680fbd4b2e7c4c47a90d2'),
 'climate_fire': []}
{'Air Temperature(Celcius)': 22,
 'Date': '2017-02-09',
 'MAX': '92.7*',
 'MIN': '64.8*',
 'Max Wind Speed': 18.1,
 'Precipitation': '0.00I',
 'Relative Humidity': 62.7,
 'Station': 948700,
 'WindSpeed  (knots)': 7.6,
 '_id': ObjectId('5af680fbd4b2e7c4c47a90d3'),
 'climate_fire': []}
{'Air Temperature(Celcius)': 31,
 'Date': '2017-02-08',
 'MAX': '96.3*',
 'MIN': 60.3,
 'Max Wind Speed': 23.9,
 'Precipitation': '0.00I',
 'Relative Humidity': 61.9,
 'Station': 948700,
 'WindSpeed  (knots)': 14.1,
 '_id': ObjectId('5af680fbd4b2e7c4c47a90d4'),
 'climate_fire': []}
{'Air Temperature(Celcius)': 21,
 'Date': '2017-02-10',
 'MAX': '78.8*',
 'MIN': 64.4,
 'Max Wind Speed': 12,
 'Precipitation': '0.00I',
 'Relative Humidity': 60.4,
 'Station': 948700,
 'WindSpeed  (knots)': 6.8,
 '_id': ObjectId('5af680fbd4b2e7c4c47a90d5'),
 'climate_fire': []}
{'Air Temperature(Celcius)': 16,
 'Date': '2017-02-12',


                   'Surface Temperature (Celcius)': 48,
                   'Surface Temperature (kelvin)': 321.8,
                   '_id': ObjectId('5af68101d4b2e7c4c47a9c84')},
                  {'Confidence': 54,
                   'Date': '2017-03-10',
                   'Datetime': '2017-03-10T04:42:30',
                   'Latitude': -37.2197,
                   'Longitude': 147.9621,
                   'Power': 17.4,
                   'Surface Temperature (Celcius)': 43,
                   'Surface Temperature (kelvin)': 316.6,
                   '_id': ObjectId('5af68101d4b2e7c4c47a9c86')}]}
{'Air Temperature(Celcius)': 20,
 'Date': '2017-03-11',
 'MAX': '78.6*',
 'MIN': 61.9,
 'Max Wind Speed': 15.9,
 'Precipitation': '0.01G',
 'Relative Humidity': 59.7,
 'Station': 948701,
 'WindSpeed  (knots)': 8.6,
 '_id': ObjectId('5af680fbd4b2e7c4c47a90f2'),
 'climate_fire': []}
{'Air Temperature(Celcius)': 21,
 'Date': '2017-03-12',
 'MAX': '78.6*',
 'MIN': '60.3*',
 'Max Wind Speed': 1

                   'Surface Temperature (kelvin)': 321.7,
                   '_id': ObjectId('5af68101d4b2e7c4c47a9c67')},
                  {'Confidence': 52,
                   'Date': '2017-03-15',
                   'Datetime': '2017-03-15T00:42:50',
                   'Latitude': -37.4075,
                   'Longitude': 147.0233,
                   'Power': 13.3,
                   'Surface Temperature (Celcius)': 43,
                   'Surface Temperature (kelvin)': 316.2,
                   '_id': ObjectId('5af68101d4b2e7c4c47a9c68')},
                  {'Confidence': 51,
                   'Date': '2017-03-15',
                   'Datetime': '2017-03-15T00:42:40',
                   'Latitude': -37.002,
                   'Longitude': 148.2055,
                   'Power': 16,
                   'Surface Temperature (Celcius)': 44,
                   'Surface Temperature (kelvin)': 317.3,
                   '_id': ObjectId('5af68101d4b2e7c4c47a9c69')},
                  {'Conf

                   'Date': '2017-03-19',
                   'Datetime': '2017-03-19T04:33:50',
                   'Latitude': -37.4336,
                   'Longitude': 147.0534,
                   'Power': 18.8,
                   'Surface Temperature (Celcius)': 51,
                   'Surface Temperature (kelvin)': 324.9,
                   '_id': ObjectId('5af68101d4b2e7c4c47a9c4f')},
                  {'Confidence': 61,
                   'Date': '2017-03-19',
                   'Datetime': '2017-03-19T04:33:10',
                   'Latitude': -37.3911,
                   'Longitude': 147.037,
                   'Power': 13.8,
                   'Surface Temperature (Celcius)': 50,
                   'Surface Temperature (kelvin)': 323.7,
                   '_id': ObjectId('5af68101d4b2e7c4c47a9c50')},
                  {'Confidence': 53,
                   'Date': '2017-03-19',
                   'Datetime': '2017-03-19T04:33:30',
                   'Latitude': -37.9184,
         

...

 'climate_fire': [{'Confidence': 50,
                   'Date': '2017-11-28',
                   'Datetime': '2017-11-28T00:33:14',
                   'Latitude': -37.749,
                   'Longitude': 148.297,
                   'Power': 17.4,
                   'Surface Temperature (Celcius)': 42,
                   'Surface Temperature (kelvin)': 315.5,
                   '_id': ObjectId('5af68101d4b2e7c4c47a926d')}]}
{'Air Temperature(Celcius)': 28,
 'Date': '2017-11-29',
 'MAX': '95.9*',
 'MIN': 59.7,
 'Max Wind Speed': 16.9,
 'Precipitation': '0.00I',
 'Relative Humidity': 56.7,
 'Station': 948702,
 'WindSpeed  (knots)': 9.3,
 '_id': ObjectId('5af680fbd4b2e7c4c47a91f8'),
 'climate_fire': [{'Confidence': 53,
                   'Date': '2017-11-29',
                   'Datetime': '2017-11-29T13:17:24',
                   'Latitude': -37.611,
                   'Longitude': 149.277,
                   'Power': 16.1,
                   'Surface Temperature (Celcius)': 38,
         

                   'Surface Temperature (Celcius)': 42,
                   'Surface Temperature (kelvin)': 316,
                   '_id': ObjectId('5af68101d4b2e7c4c47a9253')},
                  {'Confidence': 100,
                   'Date': '2017-11-30',
                   'Datetime': '2017-11-30T12:22:15',
                   'Latitude': -37.644,
                   'Longitude': 149.233,
                   'Power': 103.4,
                   'Surface Temperature (Celcius)': 55,
                   'Surface Temperature (kelvin)': 328.7,
                   '_id': ObjectId('5af68101d4b2e7c4c47a9254')},
                  {'Confidence': 100,
                   'Date': '2017-11-30',
                   'Datetime': '2017-11-30T12:22:15',
                   'Latitude': -37.642,
                   'Longitude': 149.263,
                   'Power': 158.5,
                   'Surface Temperature (Celcius)': 65,
                   'Surface Temperature (kelvin)': 338.6,
                   '_id': Object

 'WindSpeed  (knots)': 7.4,
 '_id': ObjectId('5af680fbd4b2e7c4c47a9200'),
 'climate_fire': []}
{'Air Temperature(Celcius)': 15,
 'Date': '2017-12-07',
 'MAX': '69.3*',
 'MIN': 51.8,
 'Max Wind Speed': 13,
 'Precipitation': '1.26G',
 'Relative Humidity': 51,
 'Station': 948702,
 'WindSpeed  (knots)': 9,
 '_id': ObjectId('5af680fbd4b2e7c4c47a9201'),
 'climate_fire': []}
{'Air Temperature(Celcius)': 15,
 'Date': '2017-12-08',
 'MAX': '64.6*',
 'MIN': 54.9,
 'Max Wind Speed': 12,
 'Precipitation': '0.00G',
 'Relative Humidity': 48.1,
 'Station': 948702,
 'WindSpeed  (knots)': 7.9,
 '_id': ObjectId('5af680fbd4b2e7c4c47a9202'),
 'climate_fire': [{'Confidence': 77,
                   'Date': '2017-12-08',
                   'Datetime': '2017-12-08T13:11:44',
                   'Latitude': -36.273,
                   'Longitude': 146.155,
                   'Power': 11.5,
                   'Surface Temperature (Celcius)': 36,
                   'Surface Temperature (kelvin)': 309.3,
         

                   '_id': ObjectId('5af68101d4b2e7c4c47a9236')},
                  {'Confidence': 78,
                   'Date': '2017-12-15',
                   'Datetime': '2017-12-15T13:17:17',
                   'Latitude': -37.658,
                   'Longitude': 149.339,
                   'Power': 17.3,
                   'Surface Temperature (Celcius)': 36,
                   'Surface Temperature (kelvin)': 309.8,
                   '_id': ObjectId('5af68101d4b2e7c4c47a9237')},
                  {'Confidence': 51,
                   'Date': '2017-12-15',
                   'Datetime': '2017-12-15T13:17:17',
                   'Latitude': -37.623,
                   'Longitude': 149.323,
                   'Power': 17.8,
                   'Surface Temperature (Celcius)': 38,
                   'Surface Temperature (kelvin)': 311.9,
                   '_id': ObjectId('5af68101d4b2e7c4c47a9238')},
                  {'Confidence': 65,
                   'Date': '2017-12-15',
     

 'WindSpeed  (knots)': 5.7,
 '_id': ObjectId('5af680fbd4b2e7c4c47a9210'),
 'climate_fire': [{'Confidence': 72,
                   'Date': '2017-12-21',
                   'Datetime': '2017-12-21T04:53:29',
                   'Latitude': -38.075,
                   'Longitude': 143.76,
                   'Power': 17.6,
                   'Surface Temperature (Celcius)': 46,
                   'Surface Temperature (kelvin)': 320.1,
                   '_id': ObjectId('5af68101d4b2e7c4c47a9226')}]}
{'Air Temperature(Celcius)': 19,
 'Date': '2017-12-23',
 'MAX': '78.1*',
 'MIN': 57.2,
 'Max Wind Speed': 14,
 'Precipitation': '0.00I',
 'Relative Humidity': 53.9,
 'Station': 948702,
 'WindSpeed  (knots)': 8.3,
 '_id': ObjectId('5af680fbd4b2e7c4c47a9211'),
 'climate_fire': []}
{'Air Temperature(Celcius)': 16,
 'Date': '2017-12-24',
 'MAX': '65.7*',
 'MIN': '59.0*',
 'Max Wind Speed': 16.9,
 'Precipitation': '0.00I',
 'Relative Humidity': 47,
 'Station': 948702,
 'WindSpeed  (knots)': 12,
 '_id

In [355]:
climate_fire.drop()